# Downloading Data Based on Seeing

Let's find a dataset with the highest average value of r0 (this is bad?).
First we'll search for all unembargoed VISP data, as embargoed data is no use to us for this excercise.

Next, since we want to use the highest average $r_0$, we can have Fido sort the results and output just the useful columns.

This gives us the dataset `BEOGN`, which happens to be the same one we have been using already.
(So you should already have the ASDF file available on your machine but we'll go through the motions anyway for completeness.
Fido won't re-download an existing file anyway unless told to.)
We can download the dataset ASDF with Fido to inspect it in more detail.
Remember that this only downloads a single ASDF file with some more metadata about the dataset, not the actual science data.

Now that we have access to the FITS headers we can inspect the $r_0$ more closely, just as we did in the previous session.
Remember that `DINDEX4` is the Stokes index, so we can plot the $r_0$ for just Stokes I like so:

Now let's slice down our dataset based on the first frame where $r_0$ is high:

We can now download only these files, remember you need globus-connect-personal running for this.

Now let's plot the Stokes I data at some wavelength:

You will notice that a lot of it is missing.
This is because we have deliberately only downloaded those frames with an acceptably low $r_0$.
You may also notice though, that the `Dataset` object continues to function normally without the rest of the data.
When we try to access the data, if the file is missing then `Dataset` fills in the corresponding portions of the array with NaNs.

Since the seeing is bad for a significant contiguous portion of the data, we may simply want to discount that part and look only at the useful data.
In this case we can use the sub-dataset we made earlier:

Or of course we can make any arbitrary slice to look at whatever subset of the data we prefer.